Free[S,A] 

* Free - This is the program
* S - This is the language
* A - This is the type of a vlaue it will produce once you run the programm.


## 15 - Free Monads

Base on [this blog post](https://serokell.io/blog/introduction-to-free-monads).

Remember that for a set $A$, a free monoid is a list of $A$. Such free monoid
can be coded as the following functor:
```haskell
List a = Nil | Cons a (List a)
```

Similarly, if we have
an endofunctor $F:\mathbf{Set} \to \mathbf{Set}$, then we can think of a free
monad over $F$ as a "list" of applications of $F$.
For example, consider a functor 
```haskell
F a := Nil | Cons a
```
For a value `x :: Int`, we have that  `F(x)` is either `Nil` or `Cons Int`.
A free monad over such functor would the functor
```haskell
(Free F) a := Pure a | Free (F (Free F a))
```

Suppose we have a functor:
```haskell
data F a = One a | Two a a | Two' a a | Three Int a a a
```

We want to construct tree whose leaves have type `a` and whose
nodes are one of the "subtypes" `One`, `Two`, `Two'` or `Three`, e.g.
```
     Two
    /   \
 One   Three
  |   / / | \
  a  2 a  a Two'
            / \
           a   a
```

Let's start by creating the `F` functor. 

In [1]:
using MLStyle
@data F{a} begin
    One(::a)
    Two(::a,::a)
    Two_(::a,::a)
    Three(Int, ::a, ::a,::a)
end
fmap(f::Function, x::One)   = One(f(x._1))
fmap(f::Function, x::Two)   = Two(f(x._1),f(x._2))
fmap(f::Function, x::Two_)  = Two_(f(x._1),f(x._2))
fmap(f::Function, x::Three) = Three(x._1,f(x._2),f(x._3), f(x._4));

In [2]:
# abstract type F{a} end

# struct One{a} <: F{a}
#     _1::T where T<:a
# end
# struct Two{a} <: F{a}
#     _1::T where T<:a
#     _2::T where T<:a
# end

# Two(1,2)
# Two(1,2.0)

In [3]:
fmap(x->x+1,One(10))
fmap(x->x+1,Two(10,0))
fmap(x->x+1,Two_(10,0))
fmap(x->x+1,Three(3,0.,0.,0.))

Three{Float64}(3, 1.0, 1.0, 1.0)

Now,we want to create the tree example above. 

In [4]:
Two(One("f"), Three(2,"a","b",Two_("c","d")))

LoadError: MethodError: no method matching Three(::Int64, ::String, ::String, ::Two_{String})

[0mClosest candidates are:
[0m  Three(::Int64, ::a, ::a, [91m::a[39m) where a
[0m[90m   @[39m [35mMain[39m [90m[4mIn[1]:6[24m[39m


The code above does not work. Note that
`Three` is taking an `Int`, two arguments of type `String` and a last argument
of type `Two_`. 
This goes agains the definition of `Three(_1::Int,_2::a,_3::a,_4::a)`. 

The solution to this is creating a new functor `FreeF`, where
each value `a` is wrapped into a container `Pure`, and each
of the "subfunctors" (i.e. `FreeOne`, `FreeTwo`, `FreeTwo_`, `FreeThree`)
can receive a value either `Pure{a}` or `Free{a}`.

In the example below, we will construct the `FreeF` functor manually. Yet,
the idea is that `Free` should be a functor itself, such that `Free ∘ F`
would define `FreeF`.

The `FreeF` is a functor with an `fmap` that inherits the `fmap` from `F`.

In [5]:
# @data FreeF{a} begin
#     Pure(::a)
#     FreeOne(::Union{Pure{a},FreeF{a}})
#     FreeTwo(::Union{Pure{a},FreeF{a}},::Union{Pure{a},FreeF{a}})
#     FreeTwo_(::Union{Pure{a},FreeF{a}},::Union{Pure{a},FreeF{a}})
#     FreeThree(Int, ::Union{Pure{a},FreeF{a}},::Union{Pure{a},FreeF{a}},::Union{Pure{a},FreeF{a}})
# end

@data FreeF{a} begin
    Pure(::a)
    FreeOne(::FreeF{a})
    FreeTwo(::FreeF{a},::FreeF{a})
    FreeTwo_(::FreeF{a},::FreeF{a})
    FreeThree(Int, ::FreeF{a},::FreeF{a},::FreeF{a})
end

One(a::FreeF) = One{FreeF}(a)
Two(a::FreeF, b::FreeF) = Two{FreeF}(a,b)
Two_(a::FreeF, b::FreeF) = Two_{FreeF}(a,b)
Three(i::Int, a::FreeF, b::FreeF, c::FreeF) = Three{FreeF}(i,a,b,c)

Tree = FreeTwo(FreeOne(Pure("f")),FreeThree(2,Pure("a"),Pure("b"),FreeTwo_(Pure("c"),Pure("d"))));

In [6]:
Tree

FreeTwo{String}(FreeOne{String}(Pure{String}("f")), FreeThree{String}(2, Pure{String}("a"), Pure{String}("b"), FreeTwo_{String}(Pure{String}("c"), Pure{String}("d"))))

Let us define the `fmap` so that `FreeF` becomes a functor. 

In [7]:
unfree(x::Pure)       = x._1
unfree(x::FreeOne)    = One{FreeF}(x._1)
unfree(x::FreeTwo)    = Two{FreeF}(x._1,x._2)
unfree(x::FreeTwo_)   = Two_{FreeF}(x._1,x._2)
unfree(x::FreeThree)  = Three{FreeF}(x._1,x._2,x._3,x._4)
free(x::One)    = FreeOne(x._1)
free(x::Two)    = FreeTwo(x._1,x._2)
free(x::Two_)   = FreeTwo_(x._1,x._2)
free(x::Three)  = FreeThree(x._1,x._2,x._3,x._4)


fmap(f::Function, x::Pure)    = Pure(f(x._1))
fmap(f::Function, x::FreeF) = free(fmap(y->fmap(f,y), unfree(x)))

fmap(x->x+1,FreeTwo(Pure(10),Pure(10)))

FreeTwo{Int64}(Pure{Int64}(11), Pure{Int64}(11))

In [8]:
# Two(Pure(1),FreeOne(Pure(1)))
Two{FreeF{Int}}(Pure(1),FreeOne(Pure(1)))

Two{FreeF{Int64}}(Pure{Int64}(1), FreeOne{Int64}(Pure{Int64}(1)))

In [9]:
fmap(x->x*"_",Tree)

FreeTwo{String}(FreeOne{String}(Pure{String}("f_")), FreeThree{String}(2, Pure{String}("a_"), Pure{String}("b_"), FreeTwo_{String}(Pure{String}("c_"), Pure{String}("d_"))))

In [10]:
# foldFree(f::Function, x::Pure)  = x._1
# foldFree(f::Function, x::FreeF) = f(fmap(y->foldFree(f,y),x._1))

evalString(x::One{String}) = x._1
evalString(x::Two{String}) = x._1 * x._2
evalString(x::Two_{String}) = x._2 * x._1
evalString(x::Three{String}) = string(x._1) * x._2 * x._3 * x._4

# evalString(x::One{T}) where T <: Pure{String} = x._1
# evalString(x::Two{String}) = x._1 * x._2
# evalString(x::Two_{String}) = x._2 * x._1
# evalString(x::Three{String}) = string(x._1) * x._2 * x._3 * x._4

evalString(Three(1,"ok","ok","Ok"))
evalString(One("a"))
evalString(Two("a","b"))
evalString(Two_("a","b"))

# evalString(x::One) = x._1
# evalString(x::Two) = x._1 * x._2
# evalString(x::Two_) = x._2 * x._1
# evalString(x::Three) = string(x._1) * x._2 * x._3 * x._4

cata(alg::Function, x::Pure)  = x._1
cata(alg::Function, x::FreeF) = alg(fmap(y -> cata(alg, y), unfree(x)))

# cata(evalString,Tree)

# unfree(Tree)
# cata(evalString,FreeOne(Pure("a")))
# unfree(FreeOne(Pure("a")))

cata (generic function with 2 methods)

In [11]:
cata(evalString,Tree)
# fmap(y->unfree(y),unfree(x))
# fmap(x->x*"o",x)
# cata(evalString,unfree(x)._1)

# unfree(x)
# cata(evalString, x)
# fmap(unfree(x)

"f2abdc"

In [ ]:
# cata(evalString,Tree)
# cata(evalString,Tree)
# cata(evalString,FreeOne(Pure("a")))
x = FreeOne(Pure("a"))
# foldFree(x->unfree(evalString(x)),Tree)
# fmap(x->unfree(evalString(x)),unfree(Tree))

# fmap(evalString,unfree(x))
unfree(x)

We have constructed our `FreeF` functor. Now we must show that it has a monadic structure to it. 

In [ ]:
η(x) = Pure(x)
μ(x::Pure{<:FreeF}) = x._1
μ(x::FreeF{<:FreeF}) = free(fmap(μ, unfree(x)))
# bind(x::Pure,  f::Function) = f(x._1)
# bind(x::FreeF, f::Function) = free(fmap(y->fmap(f,y), unfree(x)))

μ(FreeOne(Pure(FreeOne(Pure("a")))))
μ(FreeTwo(Pure(FreeOne(Pure(1))),Pure(FreeOne(Pure(1)))))

In [ ]:
# example = FreeThree(Pure(1))

In [ ]:
x = FreeOne(FreeOne(Pure(1)))
# x = FreeOne(Pure(1))

unfree(x)
# μ(x)

In [ ]:
# fmap(f::Function, x::Pure)    = Pure(f(x._1))
# fmap(f::Function, x::FreeF) = free(fmap(y->fmap(f,y), unfree(x)))


# fmap(μ,unfree(FreeOne(FreeOne(Pure(1)))._1))

In [ ]:
# test(x::T) where T <: FreeF = T(Pure(1))
# concatfree(Pure(1))
# concatfree(FreeOne(Pure(10)))
# concatfree(FreeOne(Pure(10))._1)
# fmap(concatfree, FreeOne(Pure(10)))

In [ ]:
concatfree(x::Pure)  = x._1
concatfree(x::T) where T <: FreeF = T(fmap(concatfree, x._1))

# η(x) = Pure(x)
# bind(x::Pure, f::Function)  = f(x._1)
# bind(x::FreeF, f::Function) = f(x._1)
# bind(mx::Maybe, f::Function) = μ(fmap(f, mx))

In [ ]:
[[1],[2,[3]]]

Consider the following functor `ToyLangF`. 

In [ ]:
using MLStyle
@data ToyLangF{next} begin
    Add(::Int, ::next)
    Mult(::Int, ::next)
    End()
end
End() = End{Any}()

fmap(f::Function, x::Add) = Add(x._1,f(x._2))
fmap(f::Function, x::Mult)= Mult(x._1,f(x._2))
fmap(f::Function, x::End) = x

We know that `List{T}` is a functor that takes types and creates monoids over such types.
More specifically, the `List` functor takes a type `T` to a type `List{T}` that
has a free monoid structure.
Analogously, we want to define a functor `Free` that takes a functor `F` and returns
a new functor `Free{F}`, which has a free monoid structure over it... Since a monoid structure
over functors is a monad, then `Free{F}` should have `(Free{F}, η, μ)`.




In [ ]:
Mult(2,Add(10, Add(10,1)))

In [ ]:
abstract type Free end

struct Pure{a} <: Free
    _1::a
end
struct Rec{f,a} <: Free
    _1::f
    _2::Free
end

In [ ]:
# abstract type Free{F{a}} where F <: Functor end

In [ ]:
abstract type Free{f,a} end

struct Pure{f,a} <: Free{f, a}
    _1::a
end
struct Roll{f,a} <: Free{f, a}
    _1::a
end
Pure(x::a) where a = Pure{Any, a}(x)
Pure{f}(x::a) where {f,a} = Pure{f, a}(x)

# Pure(10) isa Free{<: Add, Int}
# Pure{Any,Int}(10)
@show Pure(10) isa Free
@show Pure(10) isa Free{Any, Int}
# @show Pure(10) isa Free{<:, Int}
# Free{Number, Int} <: Free{<:Real,Int}
# Pure{Number}(1)

In [ ]:
abstract type List{T} end
struct Nil{T} <: List{T} end
struct Cons{T} <: List{T}
  val::T
  next::List{T} # n.b. abstract type!
end

# List a = Nil | Cons a (List a)

In [ ]:
struct Nil end
struct Cons{T}
  val::T
  next::Union{Cons{T},Nil} # n.b. abstract type!
end
# Cons{T} ≃ Cons T (Cons T (List T) | Nil)
# 
List{T} = Union{Nil, Cons{T}} # Nil | 

In [ ]:
abstract type Free{f,a} end

struct Pure{a} <: Free{<:Any, f}
    _1::a
end
struct Roll{f,a} <: Free{f,a}
    _1::
end

In [ ]:
# # # Define the Free type
# abstract type Free end

# Define the constructors
struct Pure{a}
    _1::a
end

# struct FreeF{f, a}
#     _1::f
#     # _2::Union{Pure{a},FreeF{f,a}} # Free{f,a}
# end
struct FreeF{f, a}
    _1::Union{Pure{a},FreeF{f,a}} # Free{f,a}
end
Free{f,a} = Union{Pure{a}, FreeF{f,a}}

# struct FreeF{f, a}
#     _1::f
#     _2::Union{Pure{a},FreeF{f,a}} # Free{f,a}
# end
# Pure{a} | FreeF{f,}
# Free{f,a} = Union{Pure{a},FreeF{FreeF{f,a},a}} # == Pure{a} | FreeF{f,a}  == Pure{a} | 
# Free{f,a} = Union{Pure{a},FreeF{f,FreeF{f,a}}} # == Pure{a} | FreeF{f,

fmap(g::Function, x::Pure)  = Pure(g(x._1))
# fmap(g::Function, x::FreeF) = FreeF(fmap(input->fmap(g,input),x)

# Free{ToyLangF}

# FreeF{Add,Int}(Add,Pure(10))

In [ ]:
@data Free{f,a} begin
    Pure(::a)
    Roll(::f,::Union{Pure, Roll{f,a}})
end

In [ ]:
@data List{T} begin
    Nil()
    Cons(val::T, next::Union{Cons{T},Nil})
end

Cons(1, Cons(1,Nil{Int}()))

In [ ]:
struct Nil end
struct Cons{T}
  val::T
  next::Union{Cons{T},Nil} # n.b. abstract type!
end
List{T} = Union{Cons{T}, Nil} # Nil | Cons T List T

In [ ]:
# Pure(Add(1,"s"))
Free{Add}

In [ ]:
# using MLStyle

# # abstract type Free end
# struct Pure{a}
#     _1::a
# end
# struct FreeF{f, a}
#     _1::f
#     _2::a
# end
# fmap(g::Function, x::Pure)  = Pure(g(x._1))
# # fmap(g::Function, x::FreeF) = FreeF(fmap(input->fmap(g,input),x)

# Free{f,a} = Union{Pure{a},FreeF{f,Free{f,a}}}

# # struct FreeF{f,a}
# #     _1::Union{Pure{a},FreeF{f,FreeF{f,a}}}
# # end

# # Free{f,a} = Union{Pure{a},FreeF{f,a}}
# # # Free{f}(x::Pure{a}) where {f,a}   = Free{f,a}(x)
# # Free()


# @data ToyLangF{next} begin
#     Add(::Int, ::next)
#     Mult(::Int, ::next)
#     End()
# end
# End() = End{Any}()

# fmap(f::Function, x::Add) = Add(x._1,f(x._2))
# fmap(f::Function, x::Mult)= Mult(x._1,f(x._2))
# fmap(f::Function, x::End) = x

# ToyLang = Free{ToyLangF}
# Pure(End()) isa ToyLang

In [ ]:
# fmap(x->x+1,Add(10,Pure(1)))
# FreeF(Add(10,Pure(1)))

In [ ]:
# ToyLang(Pure(1))


In [ ]:
# # Define the Free type
# abstract type Free{T} end

# # Define the constructors
# struct Pure{A} <: Free
#     _1::A
# end

# struct FreeF{F, A} <: Free
#     _1::F
#     _2::A
# end

# # Define the fmap function for Free
# fmap(f::Function, x::Pure)  = Pure(f(x._1))
# fmap(f::Function, x::FreeF) = FreeF(x._1, f(x._2))

# # Example Functor: MyF
# struct MyF{A}
#     n::Int
#     a::A
# end

# # Define Functor instance for MyF
# fmap(f::Function, x::MyF) = MyF(x.n, f(x.a))

# # Construct a Free monad using the constructors
# function free_example()
#     return FreeF(MyF(5, Pure(10)), Pure(0))
# end

# # Evaluate the Free monad
# function evaluate(x::Free)
#     if x isa Pure
#         return x._1
#     elseif x isa FreeF
#         return x._1.n + evaluate(x._1.a)
#     end
# end

# evaluate(free_example())

In [ ]:
using MLStyle
@data ToyLangF{next} begin
    Add(::Int, ::next)
    Mult(::Int, ::next)
    End()
end

fmap(f::Function, x::Add) = Add(x._1,f(x._2))
fmap(f::Function, x::Mult)= Mult(x._1,f(x._2))
fmap(f::Function, x::End) = x

In [ ]:
struct F{T}
    _1::T
end

fmap(f::Function, x::F) = F(f(x._1))

fmap(x->x+1, F(10))

In [ ]:
using MLStyle

In [ ]:
@data Free{}

In [ ]:
# struct Pure{a}
#     _1::a
# end

In [ ]:
# struct Free{f,a}
#     _1::Union{Pure{a},Free}
# end
# Free{f,a} = Union{}


# Free{F,Int}(Pure(10))

In [ ]:
Free{f}(x::Pure{a}) where {f,a}   = Free{f,a}(x)
Free{f}(x::Free{f,a}) where {f,a} = Free{f,a}(x)

In [ ]:
Pure(1)
Free{F,Int}(Pure(1))
Free{F}(Pure(1))
Free{F,Int}(Free{F}(Pure(1)))
Free{F}(Free{F}(Pure(1)))
Free{F}(Free{F}(Pure(1)))

In [ ]:
fmap(g::Function, fx::Free{f}) where f = Free{f}(fmap(g,fx._1))
fmap(g::Function, x::Pure)  = Pure(g(x._1))

In [ ]:
fmap(x->x+1, Free{F}(Pure(1)))
fmap(x->string(x+1),Free{F}(Pure(1)))
fmap(x->string(x+1),Free{F}(Free{F}(Pure(1))))

In [ ]:
using MLStyle
@data MyF{a} begin
    MyAction(Int, ::a)
end
MyAction(x,y::a) where a = MyAction{a}(x,y)
fmap(f::Function, x::MyAction) = MyAction(x._1,f(x._2))

fmap(x->x*"#", MyAction(10,"ok"))

In [ ]:
MyAction(10,Pure(1))